# Verificação da Existência de Arquivos de Configuração em Repositórios

Este script utiliza a API do GitHub para verificar a presença de arquivos de configuração em repositórios previamente filtrados. O objetivo é identificar quais repositórios contêm determinados arquivos relacionados ao desenvolvimento de software, como `package.json`, `composer.json`, entre outros.

## Importação de Bibliotecas

- **os**: Para interagir com o sistema operacional, como criação de diretórios.
- **PyGithub (`Github`)**: Para acessar e manipular dados da API do GitHub.
- **pandas (`pd`)**: Para manipulação de dados em formato tabular.

## Autenticação na API do GitHub

- É necessário um token pessoal de acesso do GitHub para autenticar na API.
- Substitua `SEU-TOKEN` pelo seu token pessoal no código.

## Lista de Arquivos Verificados

Os seguintes arquivos de configuração serão verificados nos repositórios:

- `package.json`
- `package-lock.json`
- `composer.json`
- `composer.lock`
- `vite.config.js`
- `angular.json`
- `tsconfig.json`
- `babel.config.js`
- `postcss.config.js`
- `vue.config.js`

## Estrutura do Código

1. **Importação de Bibliotecas**
2. **Autenticação na API do GitHub**
3. **Definição de Arquivos a Verificar**
4. **Função para Verificar Arquivos**
5. **Criação de Diretório de Saída**
6. **Carregamento e Preparação do DataFrame**
7. **Verificação de Arquivos nos Repositórios**
8. **Filtragem e Salvamento dos Resultados**

## Fluxo do Código

1. **Criação do Diretório de Saída**:
   - O diretório `../Database/Resultados-Busca` é criado se não existir.

2. **Carregamento dos Repositórios**:
   - Lê o arquivo `repositories_ux_filtrados.csv` contendo os repositórios filtrados.

3. **Adição de Colunas para Arquivos**:
   - Cada arquivo de configuração recebe uma coluna no DataFrame com valores iniciais definidos como `False`.

4. **Verificação de Arquivos por Repositório**:
   - Itera sobre os repositórios no DataFrame.
   - Verifica a presença dos arquivos usando a API do GitHub.
   - Atualiza o DataFrame com os resultados.
   - **Observação:** A função `check_files_in_repository`:

a. Lista todos os arquivos no repositório.
b. Compara cada arquivo da lista `files_to_check` com os arquivos presentes no repositório.
c. Retorna um dicionário indicando se cada arquivo foi encontrado.

5. **Filtragem dos Repositórios com Arquivos**:
   - Inclui apenas os repositórios que contêm pelo menos um dos arquivos verificados.

6. **Salvar os Resultados**:
   - Salva o DataFrame filtrado no arquivo `repos_with_files.csv` no diretório de saída.

## Observações**
Esse script tem um tempo médio de 2 horas de execução.

In [4]:
import os  # Importa a biblioteca os para interações com o sistema operacional
# Importa a classe Github da biblioteca PyGithub para acessar a API do GitHub
from github import Github
import pandas as pd  # Importa a biblioteca pandas para manipulação de dados

# Autenticação na API do GitHub
g = Github("SEU-TOKEN")  # Substitua pelo seu token do GitHub para autenticação

# Lista de arquivos a serem verificados nos repositórios
files_to_check = [
    'package.json',
    'package-lock.json',
    'composer.json',
    'composer.lock',
    'vite.config.js',
    'angular.json',
    'tsconfig.json',
    'babel.config.js',
    'postcss.config.js',
    'vue.config.js'
]

# Função para verificar a presença dos arquivos no repositório


def check_files_in_repository(repo):
    # Lista todos os arquivos do repositório
    repo_files = [file.path for file in repo.get_contents("")]
    # Inicializa todos os arquivos como não encontrados
    found_files = {file: False for file in files_to_check}

    # Verifica se cada arquivo está presente no repositório
    for file in files_to_check:
        if file in repo_files:
            found_files[file] = True  # Marca o arquivo como encontrado

    return found_files  # Retorna um dicionário indicando a presença dos arquivos


# Criar o diretório de saída se não existir
# Define o diretório onde os resultados serão salvos
output_dir = '../Database/Resultados-Busca'
os.makedirs(output_dir, exist_ok=True)  # Cria o diretório se não existir

# Carregar o arquivo CSV com os repositórios encontrados
# Caminho do arquivo CSV com os repositórios
input_csv = '../Database/Resultados-Busca/repositories_ux_filtrados.csv'
repos_df = pd.read_csv(input_csv)  # Lê o arquivo CSV em um DataFrame

# Adicionar colunas para cada arquivo a ser verificado no DataFrame
for file in files_to_check:
    # Inicializa as colunas como False (não encontrados)
    repos_df[file] = False

# Processar cada repositório e verificar os arquivos
for index, row in repos_df.iterrows():
    repo_name = row['name']  # Obtém o nome do repositório no GitHub

    try:
        # Obtém o repositório usando a API do GitHub
        repo = g.get_repo(repo_name)
        # Verifica os arquivos presentes no repositório
        found_files = check_files_in_repository(repo)

        # Atualiza o DataFrame com os arquivos encontrados
        for file, found in found_files.items():
            # Marca o arquivo como encontrado no DataFrame
            repos_df.at[index, file] = found

        # Mensagem de progresso
        print(f"Arquivos verificados para o repositório: {repo_name}")
    except Exception as e:
        # Mensagem de erro se o repositório não puder ser acessado
        print(f"Erro ao acessar o repositório {repo_name}: {e}")

# Filtrar os repositórios que possuem pelo menos um dos arquivos
# Filtra o DataFrame para incluir apenas repositórios que têm arquivos verificados
filtered_df = repos_df[repos_df[files_to_check].any(axis=1)]

# Caminho completo do arquivo CSV para salvar os resultados
# Define o caminho do arquivo de saída
output_csv = os.path.join(output_dir, 'repos_with_files.csv')

# Salvar o resultado em um novo arquivo CSV
# Salva o DataFrame filtrado em um arquivo CSV, sem incluir o índice
filtered_df.to_csv(output_csv, index=False)

# Mensagem de confirmação do salvamento
print(f"Resultados salvos em {output_csv}")

Arquivos verificados para o repositório: terkelg/awesome-creative-coding
Arquivos verificados para o repositório: IBAnimatable/IBAnimatable
Arquivos verificados para o repositório: orkestral/venom
Arquivos verificados para o repositório: AnswerDotAI/RAGatouille
Arquivos verificados para o repositório: trzsz/trzsz-ssh
Arquivos verificados para o repositório: vydd/sketch
Arquivos verificados para o repositório: youxin11544/MVP-RxJava-Hybride
Arquivos verificados para o repositório: XiaoMi/hiui
Arquivos verificados para o repositório: adrianhajdin/project_crowdfunding
Arquivos verificados para o repositório: samarth-p/College-ERP
Arquivos verificados para o repositório: kardolus/chatgpt-cli
Arquivos verificados para o repositório: Auromix/ROS-LLM
Arquivos verificados para o repositório: Maximilian-Winter/llama-cpp-agent
Arquivos verificados para o repositório: adrianhajdin/project_web3_battle_game
Arquivos verificados para o repositório: justinhartman/ui-ux-design-library
Arquivos verific

Following Github server redirection from /repos/AliReza1083/Portfolio to /repositories/586077453


Arquivos verificados para o repositório: AliReza1083/Portfolio
Arquivos verificados para o repositório: zynthian/zynthian-ui
Arquivos verificados para o repositório: OpenPrinting/system-config-printer
Arquivos verificados para o repositório: InfiniTimeOrg/InfiniSim
Arquivos verificados para o repositório: MycroftAI/mycroft-gui
Arquivos verificados para o repositório: BlueMystical/EDHM_UI
Arquivos verificados para o repositório: sayantanm19/js-music-player
Arquivos verificados para o repositório: pacien/tincapp
Arquivos verificados para o repositório: dehoisted/Bat2Exe
Arquivos verificados para o repositório: mediabots/InstagramBot-GUI-Python
Arquivos verificados para o repositório: anis-marrouchi/chatpdf-gpt
Arquivos verificados para o repositório: koenderks/rcityviews
Arquivos verificados para o repositório: bredele/vomit
Arquivos verificados para o repositório: freedomofpress/sunder
Arquivos verificados para o repositório: feathersui/feathersui-openfl
Arquivos verificados para o repo

Following Github server redirection from /repos/dooboolab-community/dooboo-native-ts to /repositories/121505294


Arquivos verificados para o repositório: dooboolab-community/dooboo-native-ts
Arquivos verificados para o repositório: mattbostock/webdriver_exporter
Arquivos verificados para o repositório: SHAMSRIZWAN/MvvmKotlinJetpackCompose
Arquivos verificados para o repositório: ma-shamshiri/Pacman-Game
Arquivos verificados para o repositório: ericktatsui/Touch-Menu-Like-Android
Arquivos verificados para o repositório: emanuel-braz/one_context
Arquivos verificados para o repositório: zbai/MATLAB-Groves
Arquivos verificados para o repositório: mitchvoll/NeuroEvolutionDriver
Arquivos verificados para o repositório: xiaohong-huang/RTK-Visual-Inertial-Navigation
Arquivos verificados para o repositório: chrisbeluga/kirby-navigation
Arquivos verificados para o repositório: guangqiang-liu/react-navigation-demo
Arquivos verificados para o repositório: ZJU-FAST-Lab/Terrestrial-Aerial-Navigation
Arquivos verificados para o repositório: KaiChen1008/Sim-to-Real-Virtual-Guidance-for-Robot-Navigation
Arquivos 

...

Arquivos verificados para o repositório: adamlui/ai-apps
Arquivos verificados para o repositório: microsoft/vscode-copilot-release
Arquivos verificados para o repositório: EnviralDesign/GeoPix
Arquivos verificados para o repositório: microsoft/MixedReality-UXTools-Unreal
Arquivos verificados para o repositório: jwd-ali/Switch-Animations
Arquivos verificados para o repositório: zchee/Seti_JetBrains
Arquivos verificados para o repositório: mrk-us/sketchsheets
Arquivos verificados para o repositório: RastislavMirek/FlexColorPicker
Arquivos verificados para o repositório: evmts/tevm-monorepo
Arquivos verificados para o repositório: gregjw/ui-ux
Arquivos verificados para o repositório: hirbod/react-native-volume-manager
Arquivos verificados para o repositório: sukesh-ak/ESP32-TUX
Arquivos verificados para o repositório: sejr/react-messenger
Arquivos verificados para o repositório: KL13NT/badeel
Arquivos verificados para o repositório: mblarsen/vue-browser-acl
Arquivos verificados para o rep

Following Github server redirection from /repos/vantezzen/auto-form to /repositories/669080455


Arquivos verificados para o repositório: vantezzen/auto-form
Arquivos verificados para o repositório: airbnb/HorizonCalendar
Arquivos verificados para o repositório: Dashibase/lotion
Arquivos verificados para o repositório: q191201771/lal
Arquivos verificados para o repositório: astoilkov/jsblocks
Arquivos verificados para o repositório: casibase/casibase
Arquivos verificados para o repositório: lapce/floem
Arquivos verificados para o repositório: DormyMo/SpiderKeeper
Arquivos verificados para o repositório: Yalantis/Side-Menu.iOS
Arquivos verificados para o repositório: gregnb/mui-datatables
Arquivos verificados para o repositório: fairygui/FairyGUI-unity
Arquivos verificados para o repositório: orioncactus/pretendard
Arquivos verificados para o repositório: thedevdojo/pines
Arquivos verificados para o repositório: view-design/ViewUI
Arquivos verificados para o repositório: homebridge/homebridge-config-ui-x
Arquivos verificados para o repositório: sciter-sdk/go-sciter
Arquivos verific

Request GET /repos/Tencent/QMUI_Web/contents/ failed with 403: Forbidden
Setting next backoff to 1104.977363s


Arquivos verificados para o repositório: Tencent/QMUI_Web
Arquivos verificados para o repositório: web3ui/web3uikit
Arquivos verificados para o repositório: CzBiX/qb-web
Arquivos verificados para o repositório: nikhilpanju/FabFilter
Arquivos verificados para o repositório: veeso/termscp
Arquivos verificados para o repositório: Animatious/awesome-animation
Arquivos verificados para o repositório: quick123official/quick_redis_blog
Arquivos verificados para o repositório: varunshenoy/opendream
Arquivos verificados para o repositório: NoriSte/ui-testing-best-practices
Arquivos verificados para o repositório: liuhaopen/UnityMMO
Arquivos verificados para o repositório: capy-ui/capy
Arquivos verificados para o repositório: staltz/flux-challenge
Arquivos verificados para o repositório: Mochengvia/PanuonUI.Silver
Arquivos verificados para o repositório: marijnz/unity-toolbar-extender
Arquivos verificados para o repositório: UI-Lovelace-Minimalist/UI
Arquivos verificados para o repositório: cash